In [ ]:
# CURRENTLY WORKING IN THIS ONE
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image
import numpy as np
from random import randint
from sklearn import preprocessing as p
import gc
import cv2
import math


In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
min_max_scaler = p.MinMaxScaler()
INPUT_DATA_PATH = 'test_input_new.npy'
OUTPUT_DATA_PATH = 'test_output_new.npy'
x_input = np.load(INPUT_DATA_PATH)
y_input = np.load(OUTPUT_DATA_PATH)

x_min = x_input.min(axis=(1, 2), keepdims=True)
x_max = x_input.max(axis=(1, 2), keepdims=True)
x_input = (x_input - x_min)/(x_max-x_min)

y_min = y_input.min(axis=(1, 2), keepdims=True)
y_max = y_input.max(axis=(1, 2), keepdims=True)
y_input = (y_input - y_min)/(y_max-y_min)

num_points = np.load('test_nums.npy')
# print(x_input)

In [ ]:
# x_input[x_input < 1] = 0


# print(x_input[0][0])

In [ ]:
def threshold(tensor):
    for img in range(len(tensor)):
            for channel in range(1):
                    for h in range(100):
                            for w in range(100):
                                    if (tensor[img][channel][h][w] < 1):
                                            tensor[img][channel][h][w] = 0

In [ ]:
SPLIT_IDX = 7500
x_train = torch.tensor(x_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
x_test = torch.tensor(x_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

y_train = torch.tensor(y_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
y_test = torch.tensor(y_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x_train = x_train.to(device)
x_test = x_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

test_num_points = num_points[SPLIT_IDX:]

# print (test_num_points.shape)

# threshold(x_train)                       
# threshold(x_test)
# threshold(y_train)                       
# threshold(y_test)


train_dataset = TensorDataset(x_train,y_train) # create your datset
# train_dataloader = DataLoader(train_dataset, shuffle=True) # create your dataloader

train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True) # create your dataloader

print (len(train_dataloader))

test_dataset = TensorDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=False)

#/home/bwilab/asha_ritu/line_model/conv_network.ipynb


In [ ]:
# create a CNN
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.cnn_stack = nn.Sequential( 
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(6,6), stride=1, bias=True, padding=(2,2)), # input layer
            nn.ReLU(),

            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(11,11), stride=1, bias=True, padding=(5,5)), # input layer
            nn.ReLU(),

            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(11,11), stride=1, bias=True, padding=(5,5)), # input layer
            nn.ReLU(),

            nn.Conv2d(in_channels=10, out_channels=1, kernel_size=(6,6), padding=(3,3)), # 100 x 100
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.cnn_stack(x)
        return logits

In [ ]:
model = NeuralNetwork()
if torch.cuda.is_available():
    model.cuda()
    
learning_rate = 3e-4
weight_decay=1e-5
optimizer = torch.optim.Adam(params=model.parameters(), weight_decay=weight_decay, lr=learning_rate)
batch_size = 16
epochs = 30
# Initialize the loss function
loss_fn = nn.MSELoss()
test_losses = []

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    avg_loss = 0
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
                  
        pred = model(X)
        loss = loss_fn(pred, y)
        avg_loss += loss
        loss.backward()
        optimizer.step()
        
        # if batch % 100 == 0:
        #     print(f"loss: {loss:>7f}")
    avg_loss = avg_loss / size
    print(f"average loss: {avg_loss}")

        # if batch % 50 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
size = len(train_dataloader.dataset)
print(size)

In [286]:
# test()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    # test(500)
    # test_loop(test_dataloader, model, loss_fn)
    
print("Done!")

average loss: 2.2126047042547725e-05
Epoch 15
-------------------------------


KeyboardInterrupt: 

In [ ]:
# Model class must be defined somewhere
# model = torch.load('model_final.pt')

In [ ]:
def threshold_input(model_input):
    WHITE = 1
    BLACK = 0
    THRESHOLD = 0.5

    model_input[model_input != BLACK] = WHITE

In [ ]:
def threshold_output(model_output):
    WHITE = 1
    BLACK = 0
    THRESHOLD = 0.5
    model_output[model_output <= THRESHOLD] = BLACK

In [ ]:
def mask(thresholded_input, model_output):
    return np.logical_or(thresholded_input, model_output)

In [ ]:
def test():
    avg_outside_line = 0
    avg_missed_in_line = 0
    size = len(test_dataloader.dataset)
    i = 0
    with torch.no_grad():
        for data, target in test_dataloader:
            output = model(data)
            # print(output*255)
            for out_i in range(100):
                # mask the input onto the output to isolate the black points
                final_output = mask(np.array(data.cpu().data[out_i][0]), np.array(output.cpu().data[out_i][0]))
                # number of points the model counted
                num_total = np.count_nonzero(final_output == 0.0)
                # mask the target onto the final output to check 
                masked_target = mask(np.array(target.cpu().data[out_i])[0], final_output)
                # number of points IN THE LINE the model counted 
                num_in_line = np.count_nonzero(masked_target == 0.0)

                # ideally, 0
                num_outside_line = num_total - num_in_line
                # difference between the number of points in line the model identified and the target number of points in line. ideally, 0
                missed_in_line = test_num_points[i] - num_in_line


                avg_outside_line += num_outside_line
                avg_missed_in_line += missed_in_line

                # num_expected = test_num_points[i]
                
                # curr_diff = math.sqrt((num_counted - num_expected)**2)
                # if (out_i % 50 == 0):
                #     print(f"diff: {curr_diff}")
            
                # print(f"accuracy of this trial: {(num_dots - num_expected)/num_expected * 100}")
                # avg_accuracy += curr_diff
                i += 1
    avg_outside_line = avg_outside_line / size
    avg_missed_in_line = avg_missed_in_line / size
    print(f"Average number of points misidentified (false positives): {avg_outside_line}")
    print(f"Average number of points missed (false negatives): {avg_missed_in_line}")

In [ ]:
test()

In [ ]:
i = randint(0,100)
# i = 80
print (i)

with torch.no_grad():
    for data, target in test_dataloader:
        raw_output = model(data)    
        normOutput = (raw_output-raw_output.min())/(raw_output.max()-raw_output.min())

        # INPUT
        input_arr = np.array(data.cpu().data[i][0])
        input = Image.fromarray(input_arr * 255)
        input.show()

        # OUTPUT
        output_arr = np.array(normOutput.cpu().data[i][0])
        output = Image.fromarray(output_arr * 255)
        output.show()
    
        threshold_input(input_arr)
        threshold_output(output_arr)
        input = Image.fromarray(input_arr * 255)
        # input.show()

        masked_arr = mask(input_arr, output_arr)
        # print (masked_arr.astype(float))
        masked = Image.fromarray((masked_arr * 255).astype(np.uint8))
        # masked.show()

        num_dots = np.count_nonzero(masked_arr == 0.0)
        num_expected = test_num_points[i]


        target = Image.fromarray(np.array(target.cpu().data[i][0])*255)
        # target.show()

        print(f"num counted: {num_dots}, num expected:{num_expected}")

        break

In [ ]:
torch.save(model, 'model_final.pt')